# Calibration Tutorial - Fort Peck, MT - Unirrigated Flux Plot

## Step 2: Calibrate with PEST++

Now we see if we can improve the model's performance through calibration.

### PEST++ Resources

PEST++ has excellent documentation covering both theory and practice:

1. **The PEST Manual 4th Ed.** (Doherty, 2002): https://www.epa.gov/sites/default/files/documents/PESTMAN.PDF
2. **GMDSI tutorial notebooks**: https://github.com/gmdsi/GMDSI_notebooks
3. **PEST++ User's Manual**: https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md
4. **PEST Book** (Doherty, 2015): https://pesthomepage.org/pest-book

**Note:** We use PT-JPL ETf and SNODAS SWE for calibration - not flux observations. The flux data is reserved for validation only.

### PEST++ Installation

Install PEST++ via conda-forge (recommended):

```bash
conda install conda-forge::pestpp
```

This installs `pestpp-ies` and other PEST++ tools directly into your conda environment. Verify the installation:

```bash
pestpp-ies --version
```

**Alternative:** For manual installation from source, see the [PEST++ GitHub releases](https://github.com/usgs/pestpp/releases).

In [ ]:
import os
import sys

root = os.path.abspath('../..')
sys.path.append(root)

from swimrs.container import SwimContainer
from swimrs.calibrate.pest_builder import PestBuilder
from swimrs.swim.config import ProjectConfig
from swimrs.calibrate.run_pest import run_pst

## 1. Load Configuration

Load the project configuration and prepare paths.

In [ ]:
project = '2_Fort_Peck'
project_ws = os.path.abspath('.')

config_path = os.path.join(project_ws, '2_Fort_Peck.toml')

config = ProjectConfig()
config.read_config(config_path, project_ws)

## 2. Generate Observation Files

PEST++ needs observation files (ETf and SWE) to compare against model predictions. We export these from the SwimContainer built in the data preparation step.

The export creates per-field numpy files:
- `obs_etf_{field_id}.np`: ETf observations (PT-JPL) with irrigation mask switching
- `obs_swe_{field_id}.np`: SWE observations from SNODAS

In [ ]:
# Open container for observations export and PEST++ setup
container_path = os.path.join(project_ws, 'data', f'{project}.swim')
container = SwimContainer.open(container_path, mode='r')

# Export observation files for PEST++ calibration
# Note: obs_folder is set to {pest_run_dir}/obs in the TOML config
obs_dir = os.path.join(project_ws, 'data', 'pestrun', 'obs')
os.makedirs(obs_dir, exist_ok=True)

container.export.observations(
    output_dir=obs_dir,
    etf_model='ptjpl',
    masks=('irr', 'inv_irr'),
    irr_threshold=0.1,
)
print(f"Observation files written to {obs_dir}")

## 3. Build PEST++ Control Files

The `PestBuilder` class sets up everything needed for PEST++ calibration:

### Calibration Loop
1. Initialize model with initial parameter values
2. Run model, write results
3. Compare results to observations (ETf, SWE)
4. Propose new parameters
5. Repeat until convergence

### Tunable Parameters
SWIM uses 8 tunable parameters:
- **Soil water:** `aw` (available water), `rew` (readily evaporable water), `tew` (total evaporable water)
- **NDVI-Kcb relationship:** `ndvi_alpha`, `ndvi_beta`
- **Stress threshold:** `mad` (management allowable depletion)
- **Snow melt:** `swe_alpha`, `swe_beta`

In [ ]:
# PestBuilder auto-generates custom_forward_run.py using the process package
# with portable swim_input.h5 file for fully self-contained workers
builder = PestBuilder(config, container=container, use_existing=False, verbose=False)

### Build the .pst Control File

The `build_pest()` method:
- Copies project files to a `pest/` directory
- Creates the `.pst` control file
- Sets up parameter templates (`.tpl`) and instruction files (`.ins`)

**Note:** ETf observations are sparse (only on satellite capture dates). The PEST builder automatically assigns weight 1.0 to valid observations and weight 0.0 to missing dates, ensuring we only calibrate against actual satellite captures.

In [ ]:
# Build the pest control file
# WARNING: This will erase any existing pest directory!
builder.build_pest(target_etf=config.etf_target_model, members=config.etf_ensemble_members)

In [ ]:
# Show the files created in the pest directory
pest_files = [f for f in sorted(os.listdir(builder.pest_dir)) 
              if os.path.isfile(os.path.join(builder.pest_dir, f))]
print("Files in pest directory:")
for f in pest_files:
    print(f"  {f}")

### Examine the .pst File

The PEST++ version 2 control file is concise, delegating details to external files:

In [ ]:
with open(builder.pst_file, 'r') as f:
    print(f.read())

## 4. Configure and Test

Now we:
1. Build the localizer matrix (links parameters to relevant observations)
2. Run spinup to save initial water balance state
3. Do a dry run to verify everything works
4. Set control parameters for the full calibration

In [ ]:
# Build localizer matrix
# - SWE observations update only swe_alpha and swe_beta
# - ETf observations update all other parameters
builder.build_localizer()

# Run spinup to save water balance state as initial conditions for calibration runs
# This runs the model once and saves the final state to spinup.json
print("Running spinup...")
builder.spinup(overwrite=True)

# Run a minimal model run to verify the setup
print("\nRunning dry run...")
builder.dry_run()

# Configure for 3 optimization iterations with 20 realizations each
# Increase realizations (e.g., 100-200) for production runs
builder.write_control_settings(noptmax=3, reals=20)

In [ ]:
# Show the updated control file with calibration settings
with open(builder.pst_file, 'r') as f:
    print(f.read())

## 5. Run PEST++ Calibration

Now we launch PEST++ with parallel workers. Adjust `workers` to match your machine's capabilities.

**Important:** This can take a while. Watch for the progress indicator showing completed runs (C), failed (F), and total (T).

In [ ]:
workers = config.workers or 6
_pst = f"{config.project_name}.pst"

print(f"Starting PEST++ with {workers} workers...")
print(f"Control file: {_pst}")

run_pst(builder.pest_dir,
        'pestpp-ies',
        _pst,
        num_workers=workers,
        worker_root=builder.workers_dir,
        master_dir=builder.master_dir,
        cleanup=True,
        verbose=True)

## 6. Check Results

After successful calibration, parameter files are saved for each optimization iteration:

In [ ]:
# Parameter files are in master_dir after calibration
import os
import shutil

results_dir = builder.master_dir if os.path.exists(builder.master_dir) else builder.pest_dir

par_files = [f for f in sorted(os.listdir(results_dir)) if '.par.csv' in f]
print(f"Parameter files (in {results_dir}):")
for f in par_files:
    print(f"  {f}")

if par_files:
    final_par = par_files[-1]
    print(f"\nFinal calibrated parameters: {final_par}")
    
    # Copy final parameters to pest/archive/ for consistency with calibration.py script
    archive_dir = os.path.join(builder.pest_dir, 'archive')
    os.makedirs(archive_dir, exist_ok=True)
    src = os.path.join(results_dir, final_par)
    dst = os.path.join(archive_dir, final_par)
    shutil.copy2(src, dst)
    print(f"Archived to: {dst}")

# Close the container
container.close()
print("\nContainer closed.")

## Summary

You've set up and run a PEST++ calibration using:
- PT-JPL ETf observations from Landsat (via OpenET)
- SNODAS SWE observations
- Iterative Ensemble Smoother (pestpp-ies)

The parameter files (e.g., `2_Fort_Peck.3.par.csv`) contain the calibrated parameter sets. Each row represents a different realization from the ensemble.

**Next step:** In notebook `03_calibrated_model.ipynb`, we'll run the model in forecast mode with calibrated parameters and validate against flux tower observations.